In [75]:
from imblearn import combine
import pandas as pd
import os
import numpy as np
import re
from functools import reduce

pd.set_option('display.max_columns', 50)
from IPython.display import display

from sklearn.ensemble import GradientBoostingClassifier as gbc
from sklearn.ensemble import partial_dependence as partdep

In [28]:
files = [y.group(0) for y in [re.match(r"^h\_LMOCGPTP\_(?!changepoint).+",x) for x in os.listdir("./data/failure_files/")] if y is not None] 
datasets = []
for f in files:
    datasets.append(pd.read_csv("./data/failure_files/" + f))
dataset = reduce(lambda x,y: pd.concat([x,y],axis=0),datasets)
del datasets

In [43]:
dataset.describe()

,CardType,NumSampleSecs,PmType,PmdAve,PmdMax,PmdMin,RxEdfaLbcAve,RxEdfaLbcMax,RxEdfaLbcMin,RxEdfaOprAve,RxEdfaOprMax,RxEdfaOprMin,RxEdfaOptAve,RxEdfaOptMax,RxEdfaOptMin,SoPmdAve,SoPmdMax,SoPmdMin,TxEdfaLbcAve,TxEdfaLbcMax,TxEdfaLbcMin,TxEdfaOprAve,TxEdfaOprMax,TxEdfaOprMin,TxEdfaOptAve,TxEdfaOptMax,TxEdfaOptMin,chassis_id,validity,fail
count,130261.000000,130261.0,130261.0,48478.000000,48478.000000,48478.000000,127570.000000,127570.000000,127471.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,48478.000000,48478.000000,48478.000000,127605.000000,127605.000000,127604.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000,130261.000000
mean,69.031698,900.0,15.0,39.247151,66.023187,21.774225,108.132163,108.637128,107.768572,-5.195477,-5.117125,-5.275715,8.715884,8.774083,8.657789,11413.998544,24386.230663,5595.942239,346.778301,347.187871,346.369435,-12.652105,-12.635133,-12.669654,16.353915,16.361725,16.345577,7.871757,2.938807,0.000368
std,3.881045,0.0,0.0,27.221129,38.477645,19.802402,30.619185,30.702060,30.496669,7.543309,7.520604,7.613402,9.555458,9.498401,9.659861,6024.117668,14242.712265,2778.784213,49.233424,49.245465,49.248977,6.140968,6.143308,6.139646,10.316447,10.317523,10.317084,4.515623,0.424069,0.019193
min,66.000000,900.0,15.0,-99.900000,-99.900000,-99.900000,0.002910,0.375000,0.749000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,-99.900000,-99.900000,-99.900000,192.400000,192.600000,192.220000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,1.000000,0.000000,0.000000
25%,66.000000,900.0,15.0,16.860000,29.260000,9.400000,84.680000,85.060000,84.680000,-5.440000,-5.390000,-5.490000,8.730000,8.750000,8.700000,5802.142500,11943.065000,3054.110000,321.040000,321.870000,320.370000,-11.620000,-11.610000,-11.640000,17.590000,17.600000,17.580000,5.000000,3.000000,0.000000
50%,66.000000,900.0,15.0,38.090000,70.980000,19.830000,98.550000,98.920000,98.170000,-4.460000,-4.390000,-4.510000,9.760000,9.780000,9.730000,10686.210000,22034.055000,5396.125000,345.820000,346.230000,345.100000,-11.610000,-11.590000,-11.630000,17.880000,17.880000,17.880000,8.000000,3.000000,0.000000
75%,74.000000,900.0,15.0,67.430000,100.290000,31.530000,128.550000,128.900000,127.770000,-2.990000,-2.900000,-3.080000,11.240000,11.270000,11.200000,15923.670000,34477.632500,7686.387500,368.710000,369.460000,367.960000,-11.600000,-11.580000,-11.610000,18.280000,18.280000,18.280000,10.000000,3.000000,0.000000
max,74.000000,900.0,15.0,99.360000,688.860000,73.110000,204.210000,256.670000,203.840000,-0.234000,1.610000,-0.439000,14.470000,16.180000,14.290000,29946.190000,766264.880000,15488.060000,483.780000,485.620000,482.620000,-11.390000,-11.370000,-11.410000,18.640000,18.830000,18.640000,17.000000,3.000000,1.000000


In [32]:
nodes = dataset["node_id"].value_counts().reset_index()
nodes

,index,node_id
0,HSTSTXJVO541A2008A@MA4512510040,69193
1,DNVRCO26O70040405A@MA4012230913,31896
2,ATLNGAMAO50080604A@MA4513080111,29172


In [68]:
vars = [
"PmdAve",
"PmdMax",
"PmdMin",
"RxEdfaLbcAve",
"RxEdfaLbcMax",
"RxEdfaLbcMin",
"RxEdfaOprAve",
"RxEdfaOprMax",
"RxEdfaOprMin",
"RxEdfaOptAve",
"RxEdfaOptMax",
"RxEdfaOptMin",
"SoPmdAve",
"SoPmdMax",
"SoPmdMin",
"TxEdfaLbcAve",
"TxEdfaLbcMax",
"TxEdfaLbcMin",
"TxEdfaOprAve",
"TxEdfaOprMax",
"TxEdfaOprMin",
"TxEdfaOptAve",
"TxEdfaOptMax",
"TxEdfaOptMin"
]
samp = combine.SMOTEENN(random_state=5,ratio="minority")

In [69]:
for n in [nodes.loc[0,"index"]]:
    trainset = dataset["node_id"] != n
    testset = dataset["node_id"] == n
    
    train = dataset.loc[trainset,vars]
    ytrain = dataset.loc[trainset,"fail"]
    train,ytrain = samp.fit_sample(train,ytrain)
    print(type(ytrain))

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [76]:
display(train.describe())
train

,PmdAve,PmdMax,PmdMin,RxEdfaLbcAve,RxEdfaLbcMax,RxEdfaLbcMin,RxEdfaOprAve,RxEdfaOprMax,RxEdfaOprMin,RxEdfaOptAve,RxEdfaOptMax,RxEdfaOptMin,SoPmdAve,SoPmdMax,SoPmdMin,TxEdfaLbcAve,TxEdfaLbcMax,TxEdfaLbcMin,TxEdfaOprAve,TxEdfaOprMax,TxEdfaOprMin,TxEdfaOptAve,TxEdfaOptMax,TxEdfaOptMin
count,19462.000000,19462.000000,19462.000000,58377.000000,58377.000000,58293.000000,61068.000000,61068.000000,61068.000000,61068.000000,61068.000000,61068.000000,19462.000000,19462.000000,19462.000000,58412.000000,58412.000000,58411.000000,61068.000000,61068.000000,61068.000000,61068.00000,61068.000000,61068.000000
mean,40.255591,72.687981,21.094485,89.984082,90.372668,89.743935,-7.642593,-7.554600,-7.738965,5.955537,6.032523,5.875136,14022.423491,30564.997603,6732.344040,345.610388,345.985206,345.229600,-13.712306,-13.695032,-13.730455,14.62356,14.630895,14.615542
std,22.948174,36.072833,16.635059,14.670273,14.636553,14.501332,10.282643,10.260653,10.359408,13.257079,13.188487,13.378567,6886.851800,16919.127694,3196.878523,49.845004,49.859675,49.836685,8.835062,8.838559,8.833005,14.86766,14.869099,14.868719
min,-99.900000,-99.900000,-99.900000,0.002910,0.375000,0.749000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,-55.000000,-99.900000,-99.900000,-99.900000,218.830000,218.830000,218.830000,-55.000000,-55.000000,-55.000000,-55.00000,-55.000000,-55.000000
25%,25.050000,39.622500,14.500000,79.810000,79.810000,79.810000,-6.190000,-6.120000,-6.250000,8.170000,8.210000,8.160000,5698.232500,11936.332500,3065.630000,310.310000,310.630000,310.260000,-11.620000,-11.610000,-11.640000,17.59000,17.590000,17.580000
50%,42.250000,78.330000,21.500000,86.560000,86.560000,86.560000,-5.390000,-5.340000,-5.440000,8.780000,8.820000,8.770000,15604.340000,33284.470000,7426.375000,340.830000,341.920000,339.480000,-11.610000,-11.590000,-11.630000,17.87000,17.880000,17.870000
75%,45.980000,94.150000,24.810000,98.920000,99.300000,98.550000,-4.760000,-4.700000,-4.810000,9.490000,9.520000,9.470000,19427.457500,42254.520000,9140.275000,372.460000,372.460000,372.460000,-11.590000,-11.580000,-11.610000,18.28000,18.280000,18.280000
max,99.360000,688.860000,73.110000,135.060000,163.370000,134.140000,-2.580000,-1.650000,-2.660000,11.220000,12.060000,11.170000,29946.190000,766264.880000,15488.060000,483.780000,485.620000,482.620000,-11.390000,-11.370000,-11.410000,18.64000,18.830000,18.640000


,PmdAve,PmdMax,PmdMin,RxEdfaLbcAve,RxEdfaLbcMax,RxEdfaLbcMin,RxEdfaOprAve,RxEdfaOprMax,RxEdfaOprMin,RxEdfaOptAve,RxEdfaOptMax,RxEdfaOptMin,SoPmdAve,SoPmdMax,SoPmdMin,TxEdfaLbcAve,TxEdfaLbcMax,TxEdfaLbcMin,TxEdfaOprAve,TxEdfaOprMax,TxEdfaOprMin,TxEdfaOptAve,TxEdfaOptMax,TxEdfaOptMin
0,NaN,NaN,NaN,79.06,79.06,79.06,-6.14,-6.10,-6.19,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.09,-14.08,-14.11,15.21,15.21,15.20
1,NaN,NaN,NaN,79.06,79.06,79.06,-6.16,-6.11,-6.21,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.13,15.21,15.21,15.20
2,NaN,NaN,NaN,79.06,79.06,79.06,-6.17,-6.13,-6.21,8.70,8.71,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
3,NaN,NaN,NaN,79.06,79.06,79.06,-6.16,-6.10,-6.22,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
4,NaN,NaN,NaN,79.06,79.06,79.06,-6.14,-6.08,-6.18,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.11,-14.09,-14.13,15.21,15.21,15.20
5,NaN,NaN,NaN,79.06,79.06,79.06,-6.16,-6.11,-6.20,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
6,NaN,NaN,NaN,79.06,79.06,79.06,-6.16,-6.10,-6.20,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
7,NaN,NaN,NaN,79.06,79.06,79.06,-6.15,-6.10,-6.20,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
8,NaN,NaN,NaN,79.06,79.06,79.06,-6.14,-6.08,-6.18,8.71,8.72,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.13,15.21,15.21,15.20
9,NaN,NaN,NaN,79.06,79.06,79.06,-6.13,-6.08,-6.17,8.71,8.73,8.70,NaN,NaN,NaN,260.05,260.05,260.05,-14.10,-14.08,-14.12,15.21,15.21,15.20
